# maude-magic

> Fill in a module description here

**Refs:**
- [pexpect](https://pexpect.readthedocs.io/en/stable/)
- [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html)
- [IPython magics](https://ipython.readthedocs.io/en/stable/config/custommagics.html)
- See persistent Shells.ipynb  

In [1]:
%%bash 
for package in pexpect nbdev jupyterlab-quarto
do  
    pip list | grep   "^$package *" > /dev/null && echo "$package found" || pip install $package
done
# This must be done in terminal
# nbdev_install_quarto

pexpect found
nbdev found
jupyterlab-quarto found


## The Maude Interpreter class

In [8]:
#| default_exp maude-magic

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
import os
import pexpect
import time

In [66]:
#| export
timeout = 3

class TimeoutException(Exception):
    """Exception raised if time-out."""

    def __init__(self):
        super().__init__("Timeout exception.")
        self.error_code = 1

    def __str__(self):
        return f"{self.message} (Error Code: {self.error_code})"    


class MaudeInterpreter:
    """Controls maude execution, executing commands and print responses.
       Preserve sessions between different cell executions."""
    def __init__(self,debug=False,timeout=timeout):
        """ Init variables and spawns maude. """
        self.line_count = 0
        self.debug,self.timeout = debug,timeout
        # Execute maude in environment "env". Add current working directory to MAUDE_LIB
        env = dict(os.environ)
        if not 'MAUDE_LIB' in env: raise(Exception('MAUDE_LIB environmet variable not found'))
        env['MAUDE_LIB'] += ':' + os.getcwd()
        if self.debug: 
            print(f"MAUDE_LIB={env['MAUDE_LIB']}")
        # the expawned process:
        self.sh = pexpect.spawn('maude', encoding='utf-8', env=env)
        self.sh.expect('Maude> ')
        self._sync()    

    def _sync(self):
        """ We syncrhonize with maude shell requesting a pwd.""" 
        self.sh.sendline('pwd')
        self.sh.expect('pwd.*Maude> ')
            
    def __del__(self):
        if self.debug: print('Destroying Object')
        self.__call__('quit .')
             
    # Cmmand processing
    def __call__(self,command,timeout=timeout):
        # Don't terminate session by command
        if command == 'quit .': return ''
        # strip command before send
        if self.debug: print(f"<--{repr(command.strip())}")
        stripped_command=command.strip()
        self.line_count+=len(stripped_command.split('\n'))
        self.sh.sendline(stripped_command)
        self._sync()
        if self.debug: print(f"-->{repr(self.sh.before)+repr(self.sh.after)}")
        response = self.sh.before
        # filter response
        filtered = f"Line Count:{self.line_count} lines.\n"        
        for line in self.sh.before.split('\n'):
            line=line.strip()
            if line[0]!='>': filtered += line + '\n'
        return filtered
        

Creating maude interpreter:

In [67]:
maude=MaudeInterpreter(debug=False)

Load a maude module. Show it and make some reduction:

In [68]:
print(maude('load SIMPLE-NAT .'))
print(maude('show module .'))
print(maude('red s s zero .'))

Line Count:1 lines.
load SIMPLE-NAT .
Maude>

Line Count:2 lines.
show module .
fmod SIMPLE-NAT is
sort Nat .
op zero : -> Nat .
op s_ : Nat -> Nat .
op _+_ : Nat Nat -> Nat .
vars M N : Nat .
eq zero + N = N .
eq s N + M = s (N + M) .
endfm
Maude>

Line Count:3 lines.
red s s zero .
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude>



Skip `quit` command:

In [69]:
maude('quit .')

''

Closing maude session on object destroy:

In [48]:
del maude 

## The maude Magic Class

In [49]:
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

In [50]:
# This code can be put in any Python module, it does not require IPython
# itself to be running already.  It only creates the magics subclass but
# doesn't instantiate it yet.
# from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MaudeMagics(Magics):
    """Adapts Maude Shell to a IPython Magic class.Uses a owned Maude Shell.
       Cell magics are used to execute maude commands.
       Line magics are used for command line options."""  
    def __init__(self,shell):
        # Create the owned Maude Shell instance an pass it as shell
        super(MaudeMagics,self).__init__(MaudeInterpreter(debug=False)) 
        # print("On Construntor:"+str(type(self.shell)))

    @line_cell_magic
    def maude(self, line, cell=None):
        if cell is None:
            return self.shell(line)
        else:
            # We need to split cell in commands and send it one by one 
            # to remove each command echo correctly
            # print(cell)
            # remove trailing newlines 
            print(self.shell(cell))
            
            #for command in cell.split('\n'):#print(self.shell(cell))
            #    # empty commands cause timeout exception
            #    if command: print(self.shell(command)+'\n')


# In order to actually use these magics, you must register them with a
# running IPython.

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MaudeMagics)

Manually executing 'load_ipython_extension' for test purposes:

In [51]:
load_ipython_extension(get_ipython())

Now, MaudeMagic uses an owned MaudeInterpreter to run maude commands:

In [53]:
%%maude
load SIMPLE-NAT .
show module .  
red s s zero .    
red s s s zero .

Line Count:20 lines.
load SIMPLE-NAT .
Advisory: redefining module SIMPLE-NAT.
Maude> show module .
fmod SIMPLE-NAT is
sort Nat .
op zero : -> Nat .
op s_ : Nat -> Nat .
op _+_ : Nat Nat -> Nat .
vars M N : Nat .
eq zero + N = N .
eq s N + M = s (N + M) .
endfm
Maude> red s s zero .
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude> red s s s zero .
reduce in SIMPLE-NAT : s s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s s zero
Maude>



## Use cases as test

### JSON Syntax

[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/)

In [54]:
%%maude

fmod JSON is
    --- Section 3.4 Defines JSON Objects 
    protecting STRING * (op + : String String -> String to & ) .
    protecting RAT . 
        
    --- For JSON syntax we use Number, String, Boolean, null, Array and JSON    
    subsort String < Json .
    --- subsorts Rat Bool < Json .    
    sort Json .
    sorts Dict List .
    subsorts Dict List < Json .
    sorts  Pair Pair+ Pair* .
    
    subsort  String < Json .
    subsorts Pair < Pair+ < Pair* .
    subsorts Json < Json+ < Json* .    
    sorts Json+ Json* .
        
    op _:_ : String Json -> Pair [ctor prec 30] .
    op nil-Pair* : -> Pair* [ctor] .
    op nil-Json* : -> Json* [ctor] .    
    op _,_ : Pair* Pair*   -> Pair* [ctor assoc comm id: nil-Pair*] .
    op _,_ : Pair* Pair+   -> Pair+ [ditto] .
    op _,_ : Json* Json*   -> Json* [ctor id: nil-Json*] .
    op _,_ : Json* Json+   -> Json+ [ditto] .        
    op _,_ : Json+ Json*   -> Json+ [ditto] .            
    op _,_ : Json+ Json+   -> Json+ [ditto] .                
    op {_} : Pair*         -> Dict  [ctor] . 
    op [_] : Json*         -> Json  [ctor] . 
        
endfm

        

Line Count:41 lines.
fmod JSON is
Maude>



In [55]:
%%maude
red "Juan" : "Perico" .

Line Count:73 lines.
red "Juan" : "Perico" .
reduce in JSON : "Juan" : "Perico" .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Pair: "Juan" : "Perico"
Maude>



In [56]:
%%maude

red 
{
  "name" : "Manu Sporny" ,
  "homepage" : "http://manu.sporny.org/" ,
  "image" : "http://manu.sporny.org/images/manu.png"
} .

Line Count:78 lines.
red
reduce in JSON : {"name" : "Manu Sporny", "homepage" :
"http://manu.sporny.org/", "image" :
"http://manu.sporny.org/images/manu.png"} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Dict: {"homepage" : "http://manu.sporny.org/", "image" :
"http://manu.sporny.org/images/manu.png", "name" : "Manu Sporny"}
Maude>



In [57]:
%%maude 
parse "Juan" , "Perico" .


Line Count:91 lines.
parse "Juan" , "Perico" .
Json+: "Juan", "Perico"
Maude>



In [58]:
%%maude
parse "Juan",{ "id" : "perico" } .

Line Count:94 lines.
parse "Juan",{ "id" : "perico" } .
Json+: "Juan", {"id" : "perico"}
Maude>



In [59]:
%%maude 
parse [ "A", "B"] .

Line Count:97 lines.
parse [ "A", "B"] .
Json: ["A", "B"]
Maude>



In [60]:
%%maude 
parse --- Example 2: Sample JSON document
{
  "name" : "Manu Sporny",
  "homepage" : "http://manu.sporny.org/",
  "image" : "http://manu.sporny.org/images/manu.png"
} .

Line Count:100 lines.
parse --- Example 2: Sample JSON document
Dict: {"name" : "Manu Sporny", "homepage" : "http://manu.sporny.org/", "image"
: "http://manu.sporny.org/images/manu.png"}
Maude>



In [61]:
%%maude 
--- Example 4
red 
{
  "@context" : {
    "name" : "http://schema.org/name",
    "image" : {
      "@id" : "http://schema.org/image",
      "@type" : "@id"
    },
    "homepage" : {
      "@id" : "http://schema.org/url",
      "@type" : "@id"
    }
  }
} .

Line Count:109 lines.
--- Example 4
reduce in JSON : {"@context" : {"name" : "http://schema.org/name", "homepage" :
{"@id" : "http://schema.org/url", "@type" : "@id"}, "image" : {"@id" :
"http://schema.org/image", "@type" : "@id"}}} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Dict: {"@context" : {"homepage" : {"@id" : "http://schema.org/url",
"@type" : "@id"}, "image" : {"@id" : "http://schema.org/image", "@type" :
"@id"}, "name" : "http://schema.org/name"}}
Maude>



### JSON-LD BASIC

In [62]:
%%maude
fmod JSON-LD-BASIC is
    protecting JSON * (sort Pair to Entry, Pair+ to Entry+, Pair* to Entry*,
                       sort Json to Value) .

    sorts Term IRI Keyword .
    subsorts Term IRI Keyword < String .

    vars S : String . 
        
    cmb S : Keyword if substr(S,0,1) == "@" .
    cmb S : IRI     if find(S,":",0) =/= notFound .
    cmb S : Term    if find(S,":",0) == notFound /\ substr(S,0,1) =/= "@" .    

    --- The Context 
 
    sort Context_Map ,
    subsort Context_Map < Entry .    
                
    mb T:term : V:Value  : Context_Map .   
        
    sorts Context_Map+ Context_Map* .
    subsorts Context_Map < Context_Map+ < Context_Map* .
    subsort Context_Map  < Entry .    
    subsort Context_Map+ < Entry+ .    
    subsort Context_Map* < Entry* . 
    
    op _,_ : Context_Map* Context_Map* -> Context_Map* [ditto] .
    op _,_ : Context_Map* Context_Map+ -> Context_Map+ [ditto] .                    

    sort Context_Entry .
    subsort Context_Entry  < Entry .    
    
    vars P : Entry .
    cmb P : Context_Entry if "@context" : { CE*:context_entry+ } := P .  

    ---  Context semantics:    

    vars T   : Term .
    vars Iri : IRI .
    vars V   : Value . 
    vars ... : context_entry* .
        
    eq { "@context" : { T : Iri }, T : V, ...} = { "@context" : { T : Iri }, Iri : V, ...} .

    --- The Identity entry 
    
    sort Identity .
    subsort Identity < Entry .
    cmb P:Entry : Identity if "@id" : Iri:IRI := P .

    --- The Type entry

    sort Type-Entry .
    subsort Type-Entry < Entry .
    cmb P:Entry : Type-Entry if "@type" : Iri:IRI := P .

endfm        

Line Count:132 lines.
fmod JSON-LD-BASIC is
Context_Map.
Entry+.
Entry*.
Value.
context_entry*.
statement
mb T:term : V:Value : Context_Map .
CE*:context_entry+.
statement
cmb P : Context_Entry if "@context" : {CE*:context_entry+} := P .
V:
eq { "@context" : { T : Iri } , T : V <---*HERE*
statement
eq {"@context" : {T : Iri}, T : V, ...} = {"@context" : {T : Iri}, Iri : V,
...} .
Maude>



In [22]:
%%maude 
show modules .

show modules .
fmod BOOL
fmod TRUTH-VALUE
fmod BOOL-OPS
fmod TRUTH
fmod EXT-BOOL
fmod INITIAL-EQUALITY-PREDICATE
fmod NAT
fmod INT
fmod RAT
fmod FLOAT
fmod STRING
fmod CONVERSION
fmod RANDOM
fmod BOUND
fmod QID
fth TRIV
fth STRICT-WEAK-ORDER
fth STRICT-TOTAL-ORDER
fth TOTAL-PREORDER
fth TOTAL-ORDER
fth DEFAULT
fmod LIST
fmod WEAKLY-SORTABLE-LIST
fmod SORTABLE-LIST
fmod WEAKLY-SORTABLE-LIST'
fmod SORTABLE-LIST'
fmod SET
fmod LIST-AND-SET
fmod SORTABLE-LIST-AND-SET
fmod SORTABLE-LIST-AND-SET'
fmod LIST*
fmod SET*
fmod MAP
fmod ARRAY
fmod STRING-OPS
fmod NAT-LIST
fmod QID-LIST
fmod QID-SET
fmod META-TERM
fmod META-CONDITION
fmod META-STRATEGY
fmod META-MODULE
fmod META-VIEW
fmod META-LEVEL
fmod LEXICAL
mod COUNTER
mod LOOP-MODE
mod CONFIGURATION
fmod JSON
fmod JSON-LD-BASIC
fth X :: TRIV
fmod LIST{[X]}
fth X :: STRICT-WEAK-ORDER
fmod LIST{STRICT-WEAK-ORDER}
fmod LIST{STRICT-WEAK-ORDER}{[X]}
fmod LIST{STRICT-WEAK-ORDER}{[X]} * (sort NeList{STRICT-WEAK-ORDER}{X} to
NeList{X}, sort List{STRI

In [38]:
%%maude 
red "@context" .    
red "pepe" .    
red    

TIMEOUT: Timeout exceeded.
<pexpect.pty_spawn.spawn object at 0x7437e8b0c1a0>
command: /usr/local/bin/maude
args: [b'/usr/local/bin/maude']
buffer (last 100 chars): 'rewrites: 9 in 0ms cpu (0ms real) (~ rewrites/second)\r\nresult Term: "pepe"\r\n\rMaude> red\n\r\r> pwd\n\r\r> '
before (last 100 chars): 'rewrites: 9 in 0ms cpu (0ms real) (~ rewrites/second)\r\nresult Term: "pepe"\r\n\rMaude> red\n\r\r> pwd\n\r\r> '
after: <class 'pexpect.exceptions.TIMEOUT'>
match: None
match_index: None
exitstatus: None
flag_eof: False
pid: 6364
child_fd: 68
closed: False
timeout: 30
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 2000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0.05
delayafterclose: 0.1
delayafterterminate: 0.1
searcher: searcher_re:
    0: re.compile('pwd.*Maude> ')

In [ ]:
%%maude 
--- Example 4
red 
 "@context" : {
    "name" : "http://schema.org/name",
    "image" : {
      "@id" : "http://schema.org/image",
      "@type" : "@id"
    }
} .

In [ ]:
%%maude
red {
  "@context" : {
    "name" : "http://schema.org/name"
  },
  "name" : "Manu Sporny",
  "status" : "trollin'"
} .

### HTML

## TO-DO
* Remove trailibg newlines grom cell.
* A command that ads a path to MAUDE_LIB

In [25]:
#| hide
import nbdev; nbdev.nbdev_export()